# Kinetics-400の動画データのダウンロード
Kineticsは人物の動作に関する動画のデータセット  
各クラス500本ほど，各約10秒の動画が用意されている  
400種類のものと600種類のものがあるが，今回は400の，うち8種類の動画をピックアップ

arm wrestlingとbungee jumpingのvalデータを4つずつダウンロードしてくる  
- video_download
    - download.py: ダウンロードを実行するスクリプト
    - kinetics-400_val8videos.csv: ダウンロードするビデオのYoutubeIDのリスト
    - kinetics_400_label_dictionary.csv: kineticsの動作と動作IDの対応表
    - environment.yml: downloadに使用するAnaconda環境ファイル

Anacondaは使いたくないので自力で環境を整える

`pip install -U pandas youtube-dl`  
  
Ubuntuならpandas用に  
```
apt-get install -y libbz2-dev ffmpeg
pyenv install 3.6.5
pyenv global 3.6.5
```

  
上記のCSVだけこのディレクトリに持ってくる

download.pyを参考にした以下のスクリプトを実行  
どうせ8個なので並列化は行っていない

In [2]:
import glob, os, shutil, subprocess, uuid, pandas as pd
from collections import OrderedDict

In [3]:
input_csv = "./kinetics-400_val_8videos.csv"
output_dir = "../../datasets/ptca_datasets/chapter9/kinetics_videos"

In [7]:
def parse_kinetics_annotations(input_csv, ignore_is_cc=False):
    """
    Returns a parsed DataFrame.

    arguments:
    ---------
    input_csv: str
        Path to CSV file containing the following columns:
          'YouTube Identifier,Start time,End time,Class label'

    returns:
    -------
    dataset: DataFrame
        Pandas with the following columns:
            'video-id', 'start-time', 'end-time', 'label-name'
    """
    df = pd.read_csv(input_csv)
    if 'youtube_id' in df.columns:
        columns = OrderedDict([
            ('youtube_id', 'video-id'),
            ('time_start', 'start-time'),
            ('time_end', 'end-time'),
            ('label', 'label-name')])
        df.rename(columns=columns, inplace=True)
        if ignore_is_cc:
            df = df.loc[:, df.columns.tolist()[:-1]]
    return df


def create_video_folders(dataset, output_dir, tmp_dir):
    """Creates a directory for each label name in the dataset."""
    if 'label-name' not in dataset.columns:
        this_dir = os.path.join(output_dir, 'test')
        if not os.path.exists(this_dir):
            os.makedirs(this_dir)
        # I should return a dict but ...
        return this_dir
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if not os.path.exists(tmp_dir):
        os.makedirs(tmp_dir)

    label_to_dir = {}
    for label_name in dataset['label-name'].unique():
        this_dir = os.path.join(output_dir, label_name)
        if not os.path.exists(this_dir):
            os.makedirs(this_dir)
        label_to_dir[label_name] = this_dir
    return label_to_dir


def construct_video_filename(row, label_to_dir, trim_format='%06d'):
    """Given a dataset row, this function constructs the
       output filename for a given video.
    """
    basename = '%s_%s_%s.mp4' % (row['video-id'],
                                 trim_format % row['start-time'],
                                 trim_format % row['end-time'])
    if not isinstance(label_to_dir, dict):
        dirname = label_to_dir
    else:
        dirname = label_to_dir[row['label-name']]
    output_filename = os.path.join(dirname, basename)
    return output_filename


def download_clip(video_identifier, output_filename,
                  start_time, end_time,
                  tmp_dir='/tmp/kinetics',
                  num_attempts=5,
                  url_base='https://www.youtube.com/watch?v='):
    """Download a video from youtube if exists and is not blocked.

    arguments:
    ---------
    video_identifier: str
        Unique YouTube video identifier (11 characters)
    output_filename: str
        File path where the video will be stored.
    start_time: float
        Indicates the begining time in seconds from where the video
        will be trimmed.
    end_time: float
        Indicates the ending time in seconds of the trimmed video.
    """
    
    # Defensive argument checking.
    assert isinstance(video_identifier, str), 'video_identifier must be string'
    assert isinstance(output_filename, str), 'output_filename must be string'
    assert len(video_identifier) == 11, 'video_identifier must have length 11'

    
    status = False
    # Construct command line for getting the direct video link.
    tmp_filename = os.path.join(tmp_dir, '%s.mp4' % uuid.uuid4())
    command = ['youtube-dl',
               '--quiet', '--no-warnings',
               '-f', 'mp4',
               '-o', '"%s"' % tmp_filename,
               '"%s"' % (url_base + video_identifier)]
    command = ' '.join(command)
    print(command)
    attempts = 0
    
    print("tmpfilename:", tmp_filename)
    
    while True:
        try:
            output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            attempts += 1
            if attempts == num_attempts:
                return status, err.output
        else:
            break

    # Construct command to trim the videos (ffmpeg required).
    command = ['ffmpeg',
               '-i', '"%s"' % tmp_filename,
               '-ss', str(start_time),
               '-t', str(end_time - start_time),
               '-c:v', 'libx264', '-c:a', 'copy',
               '-threads', '1',
               '-loglevel', 'panic',
               '"%s"' % output_filename]
    command = ' '.join(command)
    print(command)
    try:
        output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as err:
        return status, err.output
    
    # Check if the video was successfully saved.
    status = os.path.exists(output_filename)
    os.remove(tmp_filename)
    return status, 'Downloaded'


def download_clip_wrapper(row, label_to_dir, trim_format, tmp_dir):
    """Wrapper for parallel processing purposes."""
    output_filename = construct_video_filename(row, label_to_dir, trim_format)
    clip_id = os.path.basename(output_filename).split('.mp4')[0]

    if os.path.exists(output_filename):
        status = tuple([clip_id, True, 'Exists'])
        return status

    downloaded, log = download_clip(row['video-id'], output_filename, row['start-time'], row['end-time'], tmp_dir=tmp_dir)
    status = tuple([clip_id, downloaded, log])
    return status

In [8]:
tmp_dir='./tmp/kinetics'

# Reading and parsing Kinetics.
dataset = parse_kinetics_annotations(input_csv)

# Creates folders where videos will be saved later.
label_to_dir = create_video_folders(dataset, output_dir, tmp_dir)

# Download all clips.
for i, row in dataset.iterrows():
    st = download_clip_wrapper(row, label_to_dir, '%06d', tmp_dir)
    print(st)

# Clean tmp dir.
shutil.rmtree(tmp_dir)
os.rmdir("./tmp")

youtube-dl --quiet --no-warnings -f mp4 -o "./tmp/kinetics/47fd45d5-073b-4d6c-ac9b-465680314d1e.mp4" "https://www.youtube.com/watch?v=C4lCVBZ3ux0"
tmpfilename: ./tmp/kinetics/47fd45d5-073b-4d6c-ac9b-465680314d1e.mp4
ffmpeg -i "./tmp/kinetics/47fd45d5-073b-4d6c-ac9b-465680314d1e.mp4" -ss 28 -t 10 -c:v libx264 -c:a copy -threads 1 -loglevel panic "../datasets/chapter9/kinetics_videos/arm wrestling/C4lCVBZ3ux0_000028_000038.mp4"
('C4lCVBZ3ux0_000028_000038', True, 'Downloaded')
youtube-dl --quiet --no-warnings -f mp4 -o "./tmp/kinetics/902947f8-4c0e-4425-9020-3961126682b9.mp4" "https://www.youtube.com/watch?v=ehLnj7pXnYE"
tmpfilename: ./tmp/kinetics/902947f8-4c0e-4425-9020-3961126682b9.mp4
ffmpeg -i "./tmp/kinetics/902947f8-4c0e-4425-9020-3961126682b9.mp4" -ss 27 -t 10 -c:v libx264 -c:a copy -threads 1 -loglevel panic "../datasets/chapter9/kinetics_videos/arm wrestling/ehLnj7pXnYE_000027_000037.mp4"
('ehLnj7pXnYE_000027_000037', True, 'Downloaded')
youtube-dl --quiet --no-warnings -f mp4 

## 動画データを画像データに分割
動画をフレーム画像に変換

In [10]:
# 動画が保存されたフォルダ「kinetics_videos」にある、クラスの種類とパスを取得
class_list = os.listdir(path=output_dir)
print(class_list)

# 各クラスの動画ファイルを画像ファイルに変換する
for class_list_i in (class_list):  # クラスごとのループ

    # クラスのフォルダへのパスを取得
    class_path = os.path.join(output_dir, class_list_i)

    # 各クラスのフォルダ内の動画ファイルをひとつずつ処理するループ
    for file_name in os.listdir(class_path):

        # ファイル名と拡張子に分割
        name, ext = os.path.splitext(file_name)

        # mp4ファイルでない、フォルダなどは処理しない
        if ext != '.mp4':
            continue

        # 動画ファイルを画像に分割して保存するフォルダ名を取得
        dst_directory_path = os.path.join(class_path, name)

        # 上記の画像保存フォルダがなければ作成
        if not os.path.exists(dst_directory_path):
            os.mkdir(dst_directory_path)

        # 動画ファイルへのパスを取得
        video_file_path = os.path.join(class_path, file_name)

        # ffmpegを実行させ、動画ファイルをjpgにする （高さは256ピクセルで幅はアスペクト比を変えない）
        # kineticsの動画の場合10秒になっており、大体300ファイルになる（30 frames /sec）
        cmd = 'ffmpeg -i \"{}\" -vf scale=-1:256 \"{}/image_%05d.jpg\"'.format(
            video_file_path, dst_directory_path)
        print(cmd)
        subprocess.call(cmd, shell=True)
        print('\n')

print("動画ファイルを画像ファイルに変換しました。")


['bungee jumping', 'arm wrestling']
ffmpeg -i "../datasets/chapter9/kinetics_videos/bungee jumping/zkXOcxGnUhs_000025_000035.mp4" -vf scale=-1:256 "../datasets/chapter9/kinetics_videos/bungee jumping/zkXOcxGnUhs_000025_000035/image_%05d.jpg"


ffmpeg -i "../datasets/chapter9/kinetics_videos/bungee jumping/b6yQZjPE26c_000023_000033.mp4" -vf scale=-1:256 "../datasets/chapter9/kinetics_videos/bungee jumping/b6yQZjPE26c_000023_000033/image_%05d.jpg"


ffmpeg -i "../datasets/chapter9/kinetics_videos/bungee jumping/dAeUFSdYG1I_000010_000020.mp4" -vf scale=-1:256 "../datasets/chapter9/kinetics_videos/bungee jumping/dAeUFSdYG1I_000010_000020/image_%05d.jpg"


ffmpeg -i "../datasets/chapter9/kinetics_videos/bungee jumping/TUvSX0pYu4o_000002_000012.mp4" -vf scale=-1:256 "../datasets/chapter9/kinetics_videos/bungee jumping/TUvSX0pYu4o_000002_000012/image_%05d.jpg"


ffmpeg -i "../datasets/chapter9/kinetics_videos/arm wrestling/5JzkrOVhPOw_000027_000037.mp4" -vf scale=-1:256 "../datasets/chapter9/

## KineticsからDataLoaderを作成
めんどすぎるのでここには実装しない．  
dataloader.pyをmasterから持ってきているので読んでおく  
やっていることは以下  
- make_datapath_list: 使用する画像データのパスリストを取得
- VideoTransform: 画像の前処理(リサイズ，標準化，3Dテンソルへ変換)
    - GroupResize: 画像をまとめてリサイズ
    - GroupCenter: 画像をまとめてセンタークロップ
    - GroupToTensor: 画像をまとめてテンソル化
    - GroupImgNormalize: 画像をまとめて標準化
    - Stack: 画像を1つのテンソルにまとめる(concat)
- get_label_id_dictionary: ラベル名→IDとID→ラベル名の辞書を作成
- VideoDataset: 学習，検証用テンソルの取得
    - \_\_getitem\_\_: インタフェース
        - pull_item: データ，ラベル，ラベルIDを取得 (要修正)
            - \_get_indices: 16フレームを等間隔に選び出し，番号を取得
            - \_load_imgs: 1つの動画に関わる画像をまとめてリスト化

動作確認は使用も兼ねて次のnotebookで